## Homework: Introduction



In this homework, we'll learn more about search and use Elastic Search for practice. 

> It's possible that your answers won't match exactly. If it's the case, select the closest one.


## Q1. Running Elastic 



Run Elastic Search 8.17.6, and get the cluster information. If you run it on localhost, this is how you do it:

```bash
curl localhost:9200
```

What's the `version.build_hash` value?

In [49]:
import json
from pprint import pprint
from tqdm.notebook import tqdm
import tiktoken

In [8]:
ELASTIC_SEARCH_INFO = 'elastic-search-info.json'

In [9]:
!curl localhost:9200 -o {ELASTIC_SEARCH_INFO}

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   539  100   539    0     0   175k      0 --:--:-- --:--:-- --:--:--  175k


In [12]:
with open(ELASTIC_SEARCH_INFO) as json_data:
    info = json.load(json_data)
    pprint(info)

{'cluster_name': 'docker-cluster',
 'cluster_uuid': 'AhGWmtnlSc2xe_ht356aLg',
 'name': '3b14098e9193',
 'tagline': 'You Know, for Search',
 'version': {'build_date': '2022-10-04T07:17:24.662462378Z',
             'build_flavor': 'default',
             'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73',
             'build_snapshot': False,
             'build_type': 'docker',
             'lucene_version': '9.3.0',
             'minimum_index_compatibility_version': '7.0.0',
             'minimum_wire_compatibility_version': '7.17.0',
             'number': '8.4.3'}}


In [13]:
info['version']['build_hash']

'42f05b9372a9a4a470db3b52817899b99a76ee73'

## Getting the data



Now let's get the FAQ data. You can run this snippet:

```python
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)
```

Note that you need to have the `requests` library:

```bash
pip install requests
```

In [14]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

## Q2. Indexing the data



Index the data in the same way as was shown in the course videos. Make the `course` field a keyword and the rest should be text. 

Don't forget to install the ElasticSearch client for Python:

```bash
pip install elasticsearch
```

Which function do you use for adding your data to elastic?

* `insert`
* `index`
* `put`
* `add`

In [15]:
from datetime import datetime
from elasticsearch import Elasticsearch

#del client

In [19]:
es_client = Elasticsearch("http://localhost:9200")

In [20]:
es_client.info()

ObjectApiResponse({'name': '3b14098e9193', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'AhGWmtnlSc2xe_ht356aLg', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [21]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-question"

In [22]:
# Obtener lista de índices y imprimir sus nombres
try:
    indices = es_client.cat.indices(format='json')
    for index in indices:
        print(index['index'])
except Exception as e:
    print(f'Error al obtener los índices: {e}')

In [23]:
# Create index
res_create = es_client.indices.create(index=index_name, body=index_settings)
print(res_create)

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-question'}


In [26]:
# Index document
for doc in tqdm(documents):
    #print(doc)
    es_client.index(index=index_name, body=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

Which function do you use for adding your data to elastic?

* `insert`
* **`index`** <span style="font-size: 1.2em;">⬅️</span>
* `put`
* `add`



## Q3. Searching



Now let's search in our index. 

We will execute a query "How do execute a command on a Kubernetes pod?". 

Use only `question` and `text` fields and give `question` a boost of 4, and use `"type": "best_fields"`.

What's the score for the top ranking result?

* 84.50
* 64.50
* 44.50
* 24.50

Look at the `_score` field.

In [28]:
query = "How do execute a command on a Kubernetes pod?"

In [29]:
def es_search(query, size=5, course='data-engineering-zoomcamp'):
    search_query = {
        "size": size,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": course
                    }
                }
            }
        }
    }
    response = es_client.search(index=index_name, body=search_query)
    return response

In [30]:
response = es_search(query)

In [31]:
response

ObjectApiResponse({'took': 903, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 334, 'relation': 'eq'}, 'max_score': 31.973522, 'hits': [{'_index': 'course-question', '_id': 'FOW5dJcBbjaZewPkAf8_', '_score': 31.973522, '_source': {'text': 'Install the astronomer-cosmos package as a dependency. (see Terraform example).\nMake a new folder, dbt/, inside the dags/ folder of your Composer GCP bucket and copy paste your dbt-core project there. (see example)\nEnsure your profiles.yml is configured to authenticate with a service account key. (see BigQuery example)\nCreate a new DAG using the DbtTaskGroup class and a ProfileConfig specifying a profiles_yml_filepath that points to the location of your JSON key file. (see example)\nYour dbt lineage graph should now appear as tasks inside a task group like this:', 'section': 'Course Management Form for Homeworks', 'question': 'How to run a dbt-core project as an Airflow Task Grou

In [32]:
len(response['hits']['hits'])

5

In [33]:
response['hits']['max_score']


31.973522

In [34]:
response['hits']['hits'][0]['_score']


31.973522

What's the score for the top ranking result?

* 84.50
* 64.50
* 44.50
* **24.50** <span style="font-size: 1.2em;">⬅️</span>

## Q4. Filtering



Now ask a different question: "How do copy a file to a Docker container?".

This time we are only interested in questions from `machine-learning-zoomcamp`.

Return 3 results. What's the 3rd question returned by the search engine?

* How do I debug a docker container?
* How do I copy files from a different folder into docker container’s working directory?
* How do Lambda container images work?
* How can I annotate a graph?

In [35]:
ml_course = 'machine-learning-zoomcamp'

In [36]:
ml_couse_response = es_search(query, size=3, course=ml_course)


In [37]:
ml_couse_response


ObjectApiResponse({'took': 12, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 303, 'relation': 'eq'}, 'max_score': 44.50556, 'hits': [{'_index': 'course-question', '_id': '1uW5dJcBbjaZewPkA_-9', '_score': 44.50556, '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)', 'section': '5. Deploying Machine Learning Models', 'question': 'How do I debug a docker container?', 'course': 'machine-learning-zoomcamp'}}, {'_index': 'course-question', '_id': 'Zea5dJcBbjaZewPkBQB3', '_score': 35.433445, '_source': {'text': 'Deploy and Access the Kubernetes Dashboard\nLuke', 'section': '10. Kubernetes and TensorFlow Serving', 'question': 'Kubernet

In [38]:
ml_couse_response['hits']['hits'][2]


{'_index': 'course-question',
 '_id': '9uW5dJcBbjaZewPkBP8g',
 '_score': 33.70974,
 '_source': {'text': 'You can copy files from your local machine into a Docker container using the docker cp command. Here\'s how to do it:\nIn the Dockerfile, you can provide the folder containing the files that you want to copy over. The basic syntax is as follows:\nCOPY ["src/predict.py", "models/xgb_model.bin", "./"]\t\t\t\t\t\t\t\t\t\t\tGopakumar Gopinathan',
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I copy files from a different folder into docker container’s working directory?',
  'course': 'machine-learning-zoomcamp'}}

In [39]:
ml_couse_response['hits']['hits'][2]['_source']['question']


'How do I copy files from a different folder into docker container’s working directory?'

What's the 3rd question returned by the search engine?

* How do I debug a docker container?
* ** How do I copy files from a different folder into docker container’s working directory? ** <span style="font-size: 1.2em;">⬅️</span>
* How do Lambda container images work?
* How can I annotate a graph?

## Q5. Building a prompt



Now we're ready to build a prompt to send to an LLM. 

Take the records returned from Elasticsearch in Q4 and use this template to build the context. Separate context entries by two linebreaks (`\n\n`)
```python
context_template = """
Q: {question}
A: {text}
""".strip()
```

Now use the context you just created along with the "How do copy a file to a Docker container?" question 
to construct a prompt using the template below:

```
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()
```

What's the length of the resulting prompt? (use the `len` function)

* 946
* 1446
* 1946
* 2446


In [40]:
context_template = """
Q: {question}
A: {text}
""".strip()

In [41]:
ml_couse_response['hits']['hits'][2]['_source']


{'text': 'You can copy files from your local machine into a Docker container using the docker cp command. Here\'s how to do it:\nIn the Dockerfile, you can provide the folder containing the files that you want to copy over. The basic syntax is as follows:\nCOPY ["src/predict.py", "models/xgb_model.bin", "./"]\t\t\t\t\t\t\t\t\t\t\tGopakumar Gopinathan',
 'section': '5. Deploying Machine Learning Models',
 'question': 'How do I copy files from a different folder into docker container’s working directory?',
 'course': 'machine-learning-zoomcamp'}

In [42]:
def build_context(doc):
    context_template = """
Q: {question}
A: {text}
""".strip()
    context = context_template.format(question=doc['question'], text=doc['text']).strip()
    return context

def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

    context = ""
    
    for es_doc in search_results:
        doc = es_doc['_source']
        context = context + build_context(doc) + "\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [43]:
search_results = ml_couse_response['hits']['hits']


In [44]:
prompt = build_prompt(query, search_results)


In [46]:
pprint(prompt)

("You're a course teaching assistant. Answer the QUESTION based on the CONTEXT "
 'from the FAQ database.\n'
 'Use only the facts from the CONTEXT when answering the QUESTION.\n'
 '\n'
 'QUESTION: How do execute a command on a Kubernetes pod?\n'
 '\n'
 'CONTEXT:\n'
 'Q: How do I debug a docker container?\n'
 'A: Launch the container image in interactive mode and overriding the '
 'entrypoint, so that it starts a bash command.\n'
 'docker run -it --entrypoint bash <image>\n'
 'If the container is already running, execute a command in the specific '
 'container:\n'
 'docker ps (find the container-id)\n'
 'docker exec -it <container-id> bash\n'
 '(Marcos MJD)\n'
 '\n'
 'Q: Kubernetes-dashboard\n'
 'A: Deploy and Access the Kubernetes Dashboard\n'
 'Luke\n'
 '\n'
 'Q: How do I copy files from a different folder into docker container’s '
 'working directory?\n'
 'A: You can copy files from your local machine into a Docker container using '
 "the docker cp command. Here's how to do it:\n"
 '

In [47]:
len(prompt)

1089

What's the length of the resulting prompt? (use the `len` function)

* **946** <span style="font-size: 1.2em;">⬅️</span>
* 1446
* 1946
* 2446


## Q6. Tokens



When we use the OpenAI Platform, we're charged by the number of 
tokens we send in our prompt and receive in the response.

The OpenAI python package uses `tiktoken` for tokenization:

```bash
pip install tiktoken
```

Let's calculate the number of tokens in our query: 

```python
encoding = tiktoken.encoding_for_model("gpt-4o")
```

Use the `encode` function. How many tokens does our prompt have?

* 120
* 220
* 320
* 420

Note: to decode back a token into a word, you can use the `decode_single_token_bytes` function:

```python
encoding.decode_single_token_bytes(63842)
```

In [50]:
encoding = tiktoken.encoding_for_model("gpt-4o")

In [51]:
tokens = encoding.encode(prompt)

In [58]:
decode_back_str = ""
for token in tokens:
    b = encoding.decode_single_token_bytes(token)
    s = b.decode("utf-8")
    decode_back_str += s
pprint(decode_back_str)

("You're a course teaching assistant. Answer the QUESTION based on the CONTEXT "
 'from the FAQ database.\n'
 'Use only the facts from the CONTEXT when answering the QUESTION.\n'
 '\n'
 'QUESTION: How do execute a command on a Kubernetes pod?\n'
 '\n'
 'CONTEXT:\n'
 'Q: How do I debug a docker container?\n'
 'A: Launch the container image in interactive mode and overriding the '
 'entrypoint, so that it starts a bash command.\n'
 'docker run -it --entrypoint bash <image>\n'
 'If the container is already running, execute a command in the specific '
 'container:\n'
 'docker ps (find the container-id)\n'
 'docker exec -it <container-id> bash\n'
 '(Marcos MJD)\n'
 '\n'
 'Q: Kubernetes-dashboard\n'
 'A: Deploy and Access the Kubernetes Dashboard\n'
 'Luke\n'
 '\n'
 'Q: How do I copy files from a different folder into docker container’s '
 'working directory?\n'
 'A: You can copy files from your local machine into a Docker container using '
 "the docker cp command. Here's how to do it:\n"
 '

In [60]:
# Print token numbers
len(tokens)

240

Use the `encode` function. How many tokens does our prompt have?

* 120
* **220** <span style="font-size: 1.2em;">⬅️</span>
* 320
* 420

## Bonus: generating the answer (ungraded)

Let's send the prompt to OpenAI. What's the response?  

Note: you can replace OpenAI with Ollama. See module 2.


## Bonus: calculating the costs (ungraded)

Suppose that on average per request we send 150 tokens and receive back 250 tokens.

How much will it cost to run 1000 requests?

You can see the prices [here](https://openai.com/api/pricing/)

On June 17, the prices for gpt4o are:

* Input: $0.005 / 1K tokens
* Output: $0.015 / 1K tokens

You can redo the calculations with the values you got in Q6 and Q7.


## Submit the results

* Submit your results here: https://courses.datatalks.club/llm-zoomcamp-2025/homework/hw1
* It's possible that your answers won't match exactly. If it's the case, select the closest one.